# OR Advent 2024 December 12: TSP (Kotlin)


## Dependencies

This Jupyter Notebook solves an OR Advent problem with [Timefold Solver](https://timefold.ai/open-source-solver), the open source planning solver AI. We add it as a dependency:

In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.17.0")


## Domain

We will be assigning a list of visits to a single vehicle. Each Visit has the list of all other visits ids and the distance to them.

In [2]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId

data class Visit(@PlanningId val id: Int, val distanceMap: Map<Int, Int>) {
    fun distanceTo(visit: Visit) : Int {
        return distanceMap.get(visit.id)!!;
    }
}

In [3]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable

@PlanningEntity
class Vehicle(@PlanningId var id: Int = 0) {

    @PlanningListVariable
    val visits: List<Visit> = ArrayList()
    
    fun totalTravelDistance() : Int {
        if(visits.isEmpty()) {
            return 0;
        }

        var totalTravel = 0;
        var previousVisit = visits.get(0);
        var firstVisit = previousVisit;
        
        for (nextVisit in visits) {
            totalTravel += previousVisit.distanceTo(nextVisit)
            previousVisit = nextVisit
        }
        totalTravel += previousVisit.distanceTo(firstVisit) //complete circle
        return totalTravel;
    }
}

We gather everything in a Planning Solution

In [4]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class DeliveryPlan(
    @PlanningEntityProperty
    var vehicle: Vehicle? = null,

    @ProblemFactCollectionProperty
    @ValueRangeProvider
    var visits: List<Visit>? = null,

    @PlanningScore
    var score: HardSoftScore? = null
) {
    // No-arg constructor required for Timefold
    constructor() : this(null, emptyList())
}

## Constraints

The solver takes into account hard and soft constraints:

In [5]:
import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.*
import ai.timefold.solver.core.api.score.stream.bi.BiConstraintStream
import java.time.Duration

class OrAdventConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            //soft constraints
            minimizeTraveTime(constraintFactory)
        )
    }

    private fun minimizeTraveTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Vehicle::class.java)
            .penalize(HardSoftScore.ONE_SOFT, {v -> v.totalTravelDistance()})
            .asConstraint("travelDistance")
    }
}

## Input reader

In [6]:
import java.io.File

fun readDataset(): DeliveryPlan {
    val visits = mutableListOf<Visit>()

    val input: String = File("./instance.txt").readText()
    val lines = input.lines().filter {
        it.isNotBlank() && !it.startsWith("#") && !it.startsWith("EDGE")
    }
    val firstLine : String = lines.get(0);
    val dimensions = Regex("\\d+").find(firstLine)?.value!!.toInt()

    val allDigits = Regex("\\d+").findAll(lines.drop(1).joinToString())
    val chunks = allDigits.chunked(dimensions);
    for ((index, chunk) in chunks.withIndex()) {
        val distanceMap = chunk.mapIndexed { index: Int, s: MatchResult -> index to s.value.toInt() }.toMap()
        visits.add(Visit(index, distanceMap))
    }
    return DeliveryPlan(Vehicle(), visits)
}

## Solve it

Configure and run the solver:

In [7]:
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.event.BestSolutionChangedEvent
import ai.timefold.solver.core.api.solver.event.SolverEventListener
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.config.solver.termination.TerminationConfig

// The sole purpose is to keep all solutions and timing, so we can draw a graph later.
data class EventListener(val eventList : MutableList<BestSolutionChangedEvent<DeliveryPlan>> = mutableListOf()) : SolverEventListener<DeliveryPlan>{
    override fun bestSolutionChanged(event: BestSolutionChangedEvent<DeliveryPlan>) {
        println("Best solution changed ${event.newBestScore}}")
        eventList.add(event);
    }
}

val solverFactory: SolverFactory<DeliveryPlan> = SolverFactory.create(
    SolverConfig()
        .withSolutionClass(DeliveryPlan::class.java)
        .withEntityClasses(Vehicle::class.java)
        .withConstraintProviderClass(OrAdventConstraintProvider::class.java)
        
        // Stop the solver if no better solution is found for 30 seconds.
        .withTerminationConfig(TerminationConfig().withUnimprovedSecondsSpentLimit(60L)))

val problem: DeliveryPlan = readDataset()

println("Solving the problem ...")
val solver: Solver<DeliveryPlan> = solverFactory.buildSolver()
var listener = EventListener()
solver.addEventListener(listener)
val solution: DeliveryPlan = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Best solution changed 0hard/-2861soft}
Best solution changed 0hard/-2859soft}
Best solution changed 0hard/-2854soft}
Best solution changed 0hard/-2853soft}
Best solution changed 0hard/-2852soft}
Best solution changed 0hard/-2847soft}
Best solution changed 0hard/-2845soft}
Best solution changed 0hard/-2841soft}
Best solution changed 0hard/-2840soft}
Best solution changed 0hard/-2839soft}
Best solution changed 0hard/-2836soft}
Best solution changed 0hard/-2835soft}
Best solution changed 0hard/-2833soft}
Best solution changed 0hard/-2832soft}
Best solution changed 0hard/-2831soft}
Best solution changed 0hard/-2830soft}
Best solution changed 0hard/-2827soft}
Best solution changed 0hard/-2826soft}
Best solution changed 0hard/-2824soft}
Best solution changed 0hard/-2822soft}
Best solution changed 0hard/-2820soft}
Best solution changed 0hard/-2819soft}
Best solution changed 0hard/-2818soft}
Best solution changed 0hard/-2815soft}
Best solution changed 0hard/-2814soft}
B

## Print the solution

In [8]:
val result = solution.vehicle?.visits?.map {it -> it.id}
println("Solving finished with visit order (${result}")

Solving finished with visit order ([263, 262, 273, 228, 396, 366, 221, 329, 117, 202, 176, 148, 160, 84, 55, 30, 33, 395, 403, 365, 131, 170, 414, 315, 287, 146, 145, 321, 123, 174, 98, 120, 15, 150, 76, 29, 361, 345, 62, 4, 1, 32, 249, 171, 208, 227, 119, 225, 331, 179, 126, 0, 269, 398, 364, 327, 207, 189, 86, 125, 337, 49, 109, 107, 296, 226, 348, 46, 359, 278, 271, 157, 23, 63, 35, 75, 422, 12, 317, 270, 328, 188, 186, 183, 307, 20, 347, 264, 255, 132, 57, 43, 79, 334, 229, 281, 166, 413, 58, 386, 116, 135, 67, 134, 383, 284, 247, 303, 279, 357, 258, 333, 193, 45, 427, 265, 256, 222, 111, 362, 379, 175, 190, 104, 286, 349, 332, 70, 31, 167, 61, 16, 432, 78, 387, 90, 129, 141, 13, 2, 346, 318, 316, 408, 169, 152, 417, 390, 165, 419, 103, 223, 338, 298, 250, 415, 360, 40, 295, 219, 350, 252, 130, 294, 187, 213, 89, 285, 164, 87, 204, 377, 319, 114, 288, 168, 305, 248, 236, 351, 18, 102, 341, 203, 99, 53, 267, 5, 340, 275, 110, 147, 77, 280, 14, 441, 241, 51, 356, 74, 253, 310, 352, 2

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [9]:
%use kandy

In [10]:
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf

val df = dataFrameOf(
    "bestScores" to listener.eventList.map { it.newBestScore.toLevelNumbers().get(1) }, // map soft score
    "spentTime" to listener.eventList.map { it.timeMillisSpent / 1000 }
)

plot(df) {
    line {
        x("spentTime") {
            axis.name = "Time spent in seconds"
        }
        y("bestScores") {
            axis.name = "Soft Score"
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xPAwBu"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"bestScores":[-2861.0,-2859.0,-2854.0,-2853.0,-2852.0,-2847.0,-2845.0,-2841.0,-2840.0,-2839.0,-2836.0,-2835.0,-2833.0,-2832.0,-2831.0,-2830.0,-2827.0,-2826.0,-2824.0,-2822.0,-2820.0,-2819.0,-2818.0,-2815.0,-2814.0,-2812.0,-2811.0,-2809.0,-2807.0,-2806.0,-2805.0,-2798.0,-2797.0,-2796.0,-2795.0,-2794.0,-2791.0,-2790.0,-2788.0,-2787.0,-2786.0,-2784.0,-2783.0,-2781.0,-2780.0,-2778.0,-2777.0,-2776.0,-2775.0,-2774.0,-2773.0,-2772.0,-2771.0,-2767.0,-2766.0,-2763.0,-2761.0,-2760.0,-2759.0,-2758.0,-2756.0,-2755.0,-2754.0,-2753.0,-2752.0,-2751.0,-2749.0,-2748.0,-2747.0,-2746.0,-2745.0,-2744.0,-2743.0,-2738.0],
"spentTime":[0.0,0.0,0.0,1.0,1.0,2.0,2.0,9.0,14.0,14.0,19.0,21.0,22.0,23.0,27.0,28.0,28.0,30.0,32.0,34.0,38.0,40.0,41.0,46.0,49.0,50.0,53.0,55.0,56.0,56.0,59.0,60.0,63.0,70.0,71.0,71.0,73.0,77.0,81.0,82.0,84.0,84.0,87.0,92.0,92.0,93.0,99.0,103.0,105.0,108.0,109.0,110.0,111.0,111.0,124.0,128.0,131.0,135.0,135.0,140.0,148.0,153.0,153.0,167.0,171.0,174.0,178.0,218.0,225.0,228.0,238.0,241.0,241.0,257.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Time spent in seconds",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Soft Score",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"spentTime",
"y":"bestScores"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("xPAwBu");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.31924920060518 325.5 L23.31924920060518 325.5 L23.31924920060518 320.459349593496 L23.31924920060518 307.85772357723545 L25.133976764854612 305.3373983739839 L25.133976764854612 302.8170731707314 L26.948704329104043 290.2154471544718 L26.948704329104043 285.1747967479678 L39.651797278850054 275.0934959349597 L48.7254351000972 272.57317073170725 L48.7254351000972 270.0528455284557 L57.79907292134436 262.4918699186992 L61.42852804984322 259.97154471544764 L63.24325561409265 254.9308943089436 L65.05798317834208 252.41056910569114 L72.3168934353398 249.89024390243867 L74.13162099958923 247.3699186991871 L74.13162099958923 239.8089430894306 L77.76107612808809 237.28861788617905 L81.39053125658695 232.24796747967503 L85.01998638508582 227.207317073171 L92.27889664208354 222.16666666666697 L95.9083517705824 219.6463414634145 L97.72307933483182 217.12601626016294 L106.79671715607898 209.56504065040644 L112.24089984882727 207.04471544715489 L114.05562741307669 202.00406504065086 L119.49981010582499 199.4837398373984 L123.12926523432385 194.44308943089436 L124.94399279857328 189.40243902439033 L124.94399279857328 186.88211382113877 L130.38817549132156 184.3617886178863 L132.202903055571 166.71951219512175 L137.64708574831928 164.1991869918702 L150.3501786980653 161.67886178861772 L152.16490626231473 159.15853658536616 L152.16490626231473 156.6382113821137 L155.7943613908136 149.0772357723581 L163.05327164781133 146.55691056910564 L170.31218190480902 141.5162601626016 L172.12690946905846 138.99593495935005 L175.75636459755734 136.47560975609758 L175.75636459755734 131.43495934959356 L181.20054729030562 128.914634146342 L190.27418511155278 123.87398373983797 L190.27418511155278 121.3536585365855 L192.0889126758022 116.31300813008147 L202.9772780612988 113.79268292682991 L210.2361883182965 111.27235772357744 L213.86564344679536 108.75203252032588 L219.30982613954365 106.23170731707341 L221.12455370379308 103.71138211382186 L222.93928126804252 101.19105691056939 L224.754008832

## Notice

This code isn't optimized for benchmarking or scaling.

To learn more about planning optimization, visit [docs.timefold.ai](https://docs.timefold.ai).